In [1]:
# !pip install transformers datasets torch evaluate scikit-learn 'accelerate>=0.26.0'

In [2]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
# Step 1: Loading and Preprocessing Data
def load_and_prepare_data():
    dataset = load_dataset("s-nlp/en_paradetox_toxicity")
    dataset = dataset.remove_columns('toxic')
    dataset = dataset.rename_column('comment', 'text')
    label_column = [1] * len(dataset['train'])
    dataset = dataset['train'].add_column('label', label_column)

    train_test_split = dataset.train_test_split(test_size=0.2)
    return train_test_split['train'], train_test_split['test']

def preprocess_data(tokenizer, dataset):
    def tokenize(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    return dataset.map(tokenize, batched=True)

In [4]:
# Step 2: Initializing the Model and Tokenizer
def initialize_model():
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
    return tokenizer, model

In [5]:
# Step 3: Compute Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, np.ndarray):
        logits = torch.from_numpy(logits)

    predictions = torch.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

In [6]:
# Step 4: Evaluation
def evaluate_model(trainer, test_dataset):
    results = trainer.evaluate(eval_dataset=test_dataset)
    print("Evaluation Results:", results)
    return results

In [7]:
# Step 5: Fine-Tuning
def fine_tune_model(model, tokenizer, train_dataset, test_dataset):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_steps=500,
        save_steps=1000,
        gradient_accumulation_steps=2,
        max_grad_norm=1.0
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer

In [8]:
# Step 6: Save Model and Tokenizer
def save_model_and_tokenizer(model, tokenizer, path="./toxic-text-classifier"):
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

In [9]:
# Step 7: Prediction
def predict_toxicity(model, inputs):
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        label = torch.argmax(probabilities, dim=-1)
    return label, probabilities

In [10]:
import torch

# Main Function
def main():
    # Define the device (use "mps" if available, otherwise fallback to "cpu")
    device = torch.device("mps" if torch.has_mps else "cpu")

    # Load and preprocess data
    train_dataset, test_dataset = load_and_prepare_data()
    tokenizer, model = initialize_model()

    # Move model to the selected device
    model.to(device)

    # Preprocess data and tokenize
    tokenized_train = preprocess_data(tokenizer, train_dataset)
    tokenized_test = preprocess_data(tokenizer, test_dataset)

    # Initial evaluation before fine-tuning
    trainer = Trainer(
        model=model,
        compute_metrics=compute_metrics,
        eval_dataset=tokenized_test
    )
    print("Initial Evaluation:")
    initial_results = evaluate_model(trainer, tokenized_test)

    # Fine-tune the model
    trainer = fine_tune_model(model, tokenizer, tokenized_train.select(range(1000)), tokenized_test)

    # Evaluation after fine-tuning
    print("Evaluation After Fine-Tuning:")
    fine_tuned_results = evaluate_model(trainer, tokenized_test)

    # Save the model and tokenizer
    save_model_and_tokenizer(model, tokenizer)

    # Test Predictions
    text_samples = ["I hate this so much!", "Love"]
    for text in text_samples:
        # Tokenize and move input to the correct device
        inputs = tokenizer(text, return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Run prediction
        label, probabilities = predict_toxicity(model, inputs)
        print(f"Text: {text}\nLabel: {label}\nProbabilities: {probabilities}\n")

    # Return performance results for comparison
    return initial_results, fine_tuned_results

if __name__ == "__main__":
    initial_metrics, fine_tuned_metrics = main()
    print("Initial Metrics:", initial_metrics)
    print("Fine-Tuned Metrics:", fine_tuned_metrics)

/var/folders/df/1dbg35l15nv9_w9s8vcmmk6w0000gn/T/ipykernel_30720/3446705780.py:6: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/21205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5302 [00:00<?, ? examples/s]

Initial Evaluation:


Evaluation Results: {'eval_loss': 0.6431697607040405, 'eval_model_preparation_time': 0.0005, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 388.4401, 'eval_samples_per_second': 13.649, 'eval_steps_per_second': 1.707}


/Users/sahilgupta/Desktop/ADCNS-A2/social/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.001456,1.000000,1.000000,1.000000,1.000000


Evaluation After Fine-Tuning:


Evaluation Results: {'eval_loss': 0.0014559426344931126, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 478.9632, 'eval_samples_per_second': 11.07, 'eval_steps_per_second': 2.768, 'epoch': 1.0}
Text: I hate this so much!
Label: tensor([1], device='mps:0')
Probabilities: tensor([[0.0015, 0.9985]], device='mps:0')

Text: Love
Label: tensor([1], device='mps:0')
Probabilities: tensor([[0.0020, 0.9980]], device='mps:0')

Initial Metrics: {'eval_loss': 0.6431697607040405, 'eval_model_preparation_time': 0.0005, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 388.4401, 'eval_samples_per_second': 13.649, 'eval_steps_per_second': 1.707}
Fine-Tuned Metrics: {'eval_loss': 0.0014559426344931126, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 478.9632, 'eval_samples_per_second': 11.07, 'eval_steps_per_second': 2.768, 'epoch': 1.0}
